[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Klabauterkerl/finnish-chopper/blob/main/fairseq_morfessor.ipynb)

In [ ]:
%pip install sacrebleu sentencepiece
%pip install tensorboardX
%pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
%pip install fairseq==0.12.2

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths for the mounted Google Drive
base_path = "/content/drive/MyDrive/translation_model"
dataset_path = f"{base_path}/dataset"
!mkdir -p "{dataset_path}"
data_bin_path = f"{base_path}/data-bin"
checkpoints_path = f"{base_path}/checkpoints"
logs_path = f"{base_path}/logs"

In [ ]:
# Download and extract dataset
!wget -P "{dataset_path}" https://www.statmt.org/europarl/v9/training/europarl-v9.fi-en.tsv.gz
!gunzip "{dataset_path}/europarl-v9.fi-en.tsv.gz"

In [ ]:
# Split dataset into two files, each containing one column of the original dataset
!cut -f1 {dataset_path}/europarl-v9.fi-en.tsv > {dataset_path}/europarl-v9.fi
!cut -f2 {dataset_path}/europarl-v9.fi-en.tsv > {dataset_path}/europarl-v9.en

In [ ]:
# Install Moses for preprocessing
!git clone https://github.com/moses-smt/mosesdecoder.git

# Normalize punctuation and tokenize Finnish text
!cat ${dataset_path}/europarl-v9.fi | \
mosesdecoder/scripts/tokenizer/normalize-punctuation.perl fi | \
mosesdecoder/scripts/tokenizer/tokenizer.perl -threads 8 -a -l fi \
> ${dataset_path}/europarl-v9.tok.fi

# Normalize punctuation and tokenize English text
!cat ${dataset_path}/europarl-v9.en | \
mosesdecoder/scripts/tokenizer/normalize-punctuation.perl en | \
mosesdecoder/scripts/tokenizer/tokenizer.perl -threads 8 -a -l en \
> ${dataset_path}/europarl-v9.tok.en

# Train truecaser model for Finnish
!mosesdecoder/scripts/recaser/train-truecaser.perl \
-corpus ${dataset_path}/europarl-v9.tok.fi \
-model ${dataset_path}/truecase-model.fi

# Train truecaser model for English
!mosesdecoder/scripts/recaser/train-truecaser.perl \
-corpus ${dataset_path}/europarl-v9.tok.en \
-model ${dataset_path}/truecase-model.en

# Truecase the tokenized Finnish text
!mosesdecoder/scripts/recaser/truecase.perl \
-model ${dataset_path}/truecase-model.fi \
< ${dataset_path}/europarl-v9.tok.fi \
> ${dataset_path}/europarl-v9.tok.truecase.fi

# Truecase the tokenized English text
!mosesdecoder/scripts/recaser/truecase.perl \
-model ${dataset_path}/truecase-model.en \
< ${dataset_path}/europarl-v9.tok.en \
> ${dataset_path}/europarl-v9.tok.truecase.en

# Clean the corpus
!perl mosesdecoder/scripts/training/clean-corpus-n.perl \
${dataset_path}/tokenized.tok.truecase en fi \
${dataset_path}/tokenized.tok.clean 1 50 

In [ ]:
# Install the Morfessor for Morfessor-Encoding
%pip install morfessor

# Create Morfessor directory and set its path
morfessor_path = f"{dataset_path}/morfessor"
!mkdir -p "{morfessor_path}"

# Learn Morfessor model from tokenized data
!morfessor -t {dataset_path}/tokenized.tok.clean.fi -s {morfessor_path}/model_fi.bin
!morfessor -t {dataset_path}/tokenized.tok.clean.en -s {morfessor_path}/model_en.bin
# Segment Finnish tokenized data using learned Morfessor model
!morfessor -l /content/drive/MyDrive/translation_model/dataset/morfessor/model_fi.bin -T - \
    --output-newlines --output-format "{analysis}  " --output-format-separator " --" \
    < /content/drive/MyDrive/translation_model/dataset/tokenized.tok.clean.fi > \
    /content/drive/MyDrive/translation_model/dataset/morfessor/europarl-v9.morfessor.fi
!morfessor -l /content/drive/MyDrive/translation_model/dataset/morfessor/model_en.bin -T - \
    --output-newlines --output-format "{analysis}  " --output-format-separator " --" \
    < /content/drive/MyDrive/translation_model/dataset/tokenized.tok.clean.en > \
    /content/drive/MyDrive/translation_model/dataset/morfessor/europarl-v9.morfessor.en

In [ ]:
!head -n 10000 {morfessor_path}/europarl-v9.morfessor.fi > {morfessor_path}/test.morfessor.fi
!tail -n +10001 {morfessor_path}/europarl-v9.morfessor.fi | head -n 10000 > {morfessor_path}/valid.morfessor.fi
!tail -n +20001 {morfessor_path}/europarl-v9.morfessor.fi > {morfessor_path}/train.morfessor.fi

!head -n 10000 {morfessor_path}/europarl-v9.morfessor.en > {morfessor_path}/test.morfessor.en
!tail -n +10001 {morfessor_path}/europarl-v9.morfessor.en | head -n 10000 > {morfessor_path}/valid.morfessor.en
!tail -n +20001 {morfessor_path}/europarl-v9.morfessor.en > {morfessor_path}/train.morfessor.en

In [ ]:
# Define paths for the mounted Google Drive
base_path = "/content/drive/MyDrive/translation_model"
data_bin_path = f"{base_path}/data-bin"
checkpoints_path = f"{base_path}/checkpoints"
dataset_path = f"{base_path}/dataset"

# Create directories in Google Drive
!mkdir -p "{data_bin_path}"
!mkdir -p "{checkpoints_path}"

In [ ]:
# Create Dataset using Morfessor Data
!fairseq-preprocess --source-lang fi --target-lang en \
    --trainpref {morfessor_path}/train.morfessor --validpref {morfessor_path}/valid.morfessor --testpref {dataset_path}/test.morfessor \
    --destdir {data_bin_path}/morfessor --joined-dictionary --workers 20

In [ ]:
# Train Model using Morfessor Dataset
!fairseq-train {data_bin_path}/morfessor \
    --arch transformer --share-all-embeddings \
    --encoder-layers 5 --decoder-layers 5 \
    --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --encoder-ffn-embed-dim 2048 --decoder-ffn-embed-dim 2048 \
    --encoder-attention-heads 8 --decoder-attention-heads 8 \
    --dropout 0.1 --attention-dropout 0.1 --relu-dropout 0.1 \
    --optimizer adam --lr 0.0005 --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 --warmup-init-lr 1e-07 \
    --stop-min-lr 1e-09 --clip-norm 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --weight-decay 0.0001 --max-tokens 4096 \
    --update-freq 1 --max-epoch 30 --save-interval 1 \
    --keep-last-epochs 5 --log-format simple --log-interval 100 \
    --tensorboard-logdir {logs_path} --seed 42 \
    --save-dir {checkpoints_path}/morfessor \
    --amp

In [ ]:
# Generate translations using Morfessor trained model
! fairseq-generate {data_bin_path}/morfessor \
    --path {checkpoints_path}/morfessor/checkpoint_best.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset test \
    --remove-bpe > {base_path}/translations_morfessor.txt

In [ ]:
# Compute BLEU score
!grep ^H {base_path}/translations.txt | cut -f3- > {base_path}/hyp.txt
!grep ^T {base_path}/translations.txt | cut -f2- > {base_path}/ref.txt
!mosesdecoder/scripts/generic/multi-bleu.perl {base_path}/ref.txt < {base_path}/hyp.txt